# Preparation for class in week 5


Our example is inspired by the Dutch LISS data, in particular the waves on Time Use and consumption in November 2019 and in April 2020. 

In particular, the data contain the following variables, alphabetically sorted:

| Variable     | Content                                                           |
|:-------------|:------------------------------------------------------------      |
| geslacht     | Gender (Man: Male, Vrouw: Female)                                 |
| nohouse_encr | Household identifier                                              |
| nomem_encr   | Member identifier                                                 |
| v1q1_v1col1  | Working hours (Nov) / Working hours at workplace (Apr)            |
| v1q1a_v1col1 | Working hours in home office, no kids in HH (Apr)                 |
| v1q1b_v1col1 | Working hours in home office while responsible for kids (Apr)     |
| v1q1c_v1col1 | Working hours in home office while not responsible for kids (Apr) |
| v1q5_v1col1  | Childcare hours (all in Nov, residual in Apr)                     |
| v1q5a_v1col1 | Homeschooling hours (Apr)                                         |


In this exercise, we will only work with the November data.

We start by importing Pandas

In [601]:
import pandas as pd
import numpy as np
from functools import reduce
from statistics import mean

## Read in the data

The November data is stored in Stata format. Replace the `XXXXX` in the next cell by the appropriate Pandas function.

In [602]:
nov_2019 = pd.read_stata("time_use_2019-11.dta", convert_categoricals=True)
apr_2020 = pd.read_csv("time_use_2020-04.csv")

## Browse the data

You can browse through the data by just typing the name of a DataFrame as the last thing in a cell.

This will yield a nice html-formatted output. Use this to find out the difference between setting `convert_categoricals` to `True` or `False` in the call above. In case you know some Stata: Can you explain what is happening? Else don't worry about the reasons behind what is going on.


When convert_categoricals=True then the gender column displays Man and Vroww. When False it displays 1 and 0.
I do not know the reason but it seems self-explanatory.

In [603]:
nov_2019.head()

,nomem_encr,geslacht,v1q1_v1col1,v1q5_v1col1,nohouse_encr
0,1687033.0,Man,0.0,2.0,1049420.0
1,1662353.0,Vrouw,9.0,0.0,1049420.0
2,1631191.0,Man,67.0,1.0,1011033.0
3,1687630.0,Vrouw,17.0,6.0,1011033.0
4,1746405.0,Man,40.0,12.0,1047651.0


In [604]:
apr_2020.head()

,geslacht,v1q1_v1col1,v1q1a_v1col1,v1q1b_v1col1,v1q1c_v1col1,v1q5_v1col1,v1q5a_v1col1,nohouse_encr,nomem_encr
0,Man,0.0,NaN,0.0,0.0,0.0,0.0,1049420,1687033
1,Vrouw,3.0,NaN,0.0,0.0,0.0,0.0,1049420,1662353
2,Man,47.0,NaN,4.0,0.0,4.0,0.0,1011033,1631191
3,Vrouw,8.0,NaN,0.0,8.0,0.0,0.0,1011033,1687630
4,Man,36.0,NaN,0.0,0.0,0.0,0.0,1047651,1746405


### Make sure all the columns have a sensible data type (try: apr_2020.dtypes).

In [605]:
nov_2019.dtypes

nomem_encr       float64
geslacht        category
v1q1_v1col1      float64
v1q5_v1col1      float64
nohouse_encr     float64
dtype: object

In [606]:
apr_2020.dtypes

geslacht         object
v1q1_v1col1     float64
v1q1a_v1col1    float64
v1q1b_v1col1    float64
v1q1c_v1col1    float64
v1q5_v1col1     float64
v1q5a_v1col1    float64
nohouse_encr      int64
nomem_encr        int64
dtype: object

In [607]:
# Count each categories in gender including NAs:
apr_2020['geslacht'].value_counts(dropna=False)

Man      56
Vrouw    50
Name: geslacht, dtype: int64

## Rename categories


In [608]:
# Convert gender column to categorical:
apr_2020['geslacht'] = pd.Categorical(apr_2020['geslacht'], categories=['Man', 'Vrouw'])
apr_2020['geslacht'] = apr_2020['geslacht'].cat.rename_categories({'Man': 'Male', 'Vrouw': 'Female'})
apr_2020['geslacht'].dtypes

nov_2019['geslacht'] = pd.Categorical(nov_2019['geslacht'], categories=['Man', 'Vrouw'])
nov_2019['geslacht'] = nov_2019['geslacht'].cat.rename_categories({'Man': 'Male', 'Vrouw': 'Female'})
nov_2019['geslacht'].dtypes

CategoricalDtype(categories=['Male', 'Female'], ordered=True)

In [609]:
apr_2020['geslacht'].value_counts(dropna=False)


Male      56
Female    50
Name: geslacht, dtype: int64

In [610]:
nov_2019['geslacht'].value_counts(dropna=False)

Male      46
Female    42
Name: geslacht, dtype: int64

## Rename variables

We give the Dutch and partly cryptic variable names sensible identifiers. The `replace` method on DataFrames returns a new DataFrame, so we need to assign it to `nov_2019` again if we want to continue working with it.

Note that this is a very stateful transformation if you assign to the same variable; you will not be able to successfully execute the cell twice without re-loading the data above.

In [611]:
nov_2019 = nov_2019.rename(
    columns={
        "geslacht": "gender",
        "v1q1_v1col1": "working_hours",
        "v1q5_v1col1": "childcare_hours",
        "nohouse_encr": "household_identifier",
        "nomem_encr": "member_identifier"
    }
)

apr_2020 = apr_2020.rename(
    columns={
        "geslacht": "gender",
        "nohouse_encr": "hh_id",
        "nomem_encr": "ind_id",
        "v1q1_v1col1": "working_hours_workplace",
        "v1q1a_v1col1": "working_hours_home_nokids",
        "v1q1b_v1col1": "working_hours_home_whilekids",
        "v1q1c_v1col1": "working_hours_home_while_no_kids",
        "v1q5_v1col1": "childcare_hours_res",
        "v1q5a_v1col1": "homeschool_hours"        
    }
)

Again view the dataframe to see if the changes applied.

In [612]:
nov_2019.head()

,member_identifier,gender,working_hours,childcare_hours,household_identifier
0,1687033.0,Male,0.0,2.0,1049420.0
1,1662353.0,Female,9.0,0.0,1049420.0
2,1631191.0,Male,67.0,1.0,1011033.0
3,1687630.0,Female,17.0,6.0,1011033.0
4,1746405.0,Male,40.0,12.0,1047651.0


In [613]:
apr_2020.head()

,gender,working_hours_workplace,working_hours_home_nokids,working_hours_home_whilekids,working_hours_home_while_no_kids,childcare_hours_res,homeschool_hours,hh_id,ind_id
0,Male,0.0,NaN,0.0,0.0,0.0,0.0,1049420,1687033
1,Female,3.0,NaN,0.0,0.0,0.0,0.0,1049420,1662353
2,Male,47.0,NaN,4.0,0.0,4.0,0.0,1011033,1631191
3,Female,8.0,NaN,0.0,8.0,0.0,0.0,1011033,1687630
4,Male,36.0,NaN,0.0,0.0,0.0,0.0,1047651,1746405


## Convert data types

`nomem_encr` and `nohouse_encr` are classical identifiers. They can be used to identify a particular observation, but they do not carry any meaning beyond that.

You have heard in the screencast that you should never use floating point numbers for identifiers. It is common that this happens in Stata, though (e.g., through use of `compress` or mathematical operations of integers, which do implicit type conversions unless you request an integer back).

Add new columns `hh_id` and `ind_id` with sensible data types. 


In [614]:
# Converting a panda series by using astype():
nov_2019["hh_id"] = nov_2019["household_identifier"].astype(int)
nov_2019["ind_id"] = nov_2019["member_identifier"].astype(int)


In [615]:
# Again view the dataframe
nov_2019.head()

,member_identifier,gender,working_hours,childcare_hours,household_identifier,hh_id,ind_id
0,1687033.0,Male,0.0,2.0,1049420.0,1049420,1687033
1,1662353.0,Female,9.0,0.0,1049420.0,1049420,1662353
2,1631191.0,Male,67.0,1.0,1011033.0,1011033,1631191
3,1687630.0,Female,17.0,6.0,1011033.0,1011033,1687630
4,1746405.0,Male,40.0,12.0,1047651.0,1047651,1746405


In [616]:
apr_2020.head()

,gender,working_hours_workplace,working_hours_home_nokids,working_hours_home_whilekids,working_hours_home_while_no_kids,childcare_hours_res,homeschool_hours,hh_id,ind_id
0,Male,0.0,NaN,0.0,0.0,0.0,0.0,1049420,1687033
1,Female,3.0,NaN,0.0,0.0,0.0,0.0,1049420,1662353
2,Male,47.0,NaN,4.0,0.0,4.0,0.0,1011033,1631191
3,Female,8.0,NaN,0.0,8.0,0.0,0.0,1011033,1687630
4,Male,36.0,NaN,0.0,0.0,0.0,0.0,1047651,1746405


## Select columns

We do not need to keep the old identifiers anymore.

You can select a subset of columns by including a list of column names in the standard square brackets used for indexing in Python.

Replace the XXXX and YYYY appropriately and include all other variables that you want to keep.

In [617]:
# Displaying columns' name:
nov_2019.columns.values

array(['member_identifier', 'gender', 'working_hours', 'childcare_hours',
       'household_identifier', 'hh_id', 'ind_id'], dtype=object)

In [618]:
nov_2019 = nov_2019[['hh_id', 'ind_id', "gender", "working_hours", "childcare_hours"]]

In [619]:
nov_2019.head()

,hh_id,ind_id,gender,working_hours,childcare_hours
0,1049420,1687033,Male,0.0,2.0
1,1049420,1662353,Female,9.0,0.0
2,1011033,1631191,Male,67.0,1.0
3,1011033,1687630,Female,17.0,6.0
4,1047651,1746405,Male,40.0,12.0


## Set index

The default index created by Pandas (a DataFrame always has an index) does not make much sense.

Create an index based on a column / several columns that makes sense to you via replacing the XXXX by an appropriate construct.

In [620]:
nov_2019_with_index = nov_2019.set_index(['hh_id', 'ind_id'])
apr_2020_with_index = apr_2020.set_index(['hh_id', 'ind_id'])

In [621]:
nov_2019_with_index.head(10)

gender  working_hours  childcare_hours
hh_id   ind_id                                         
1049420 1687033    Male            0.0              2.0
        1662353  Female            9.0              0.0
1011033 1631191    Male           67.0              1.0
        1687630  Female           17.0              6.0
1047651 1746405    Male           40.0             12.0
        1712561  Female            0.0             40.0
1059082 1755058    Male           32.0              8.0
        1703625  Female            0.0             40.0
1032991 1660073    Male           40.0             20.0
        1679690  Female            0.0              0.0

In [622]:
apr_2020_with_index.head(10)

gender  working_hours_workplace  working_hours_home_nokids  \
hh_id   ind_id                                                                
1049420 1687033    Male                      0.0                        NaN   
        1662353  Female                      3.0                        NaN   
1011033 1631191    Male                     47.0                        NaN   
        1687630  Female                      8.0                        NaN   
1047651 1746405    Male                     36.0                        NaN   
        1712561  Female                      0.0                        NaN   
1059082 1703625  Female                      0.0                        NaN   
        1755058    Male                      0.0                        NaN   
1032311 1705264    Male                     40.0                        NaN   
1050449 1738962    Male                      0.0                        NaN   

                 working_hours_home_whilekids  \
hh_id   ind_id                                  
1049420 1687033                           0.0   
        1662353                           0.0   
1011033 1631191                           4.0   
        1687630                           0.0   
1047651 1746405                           0.0   
        1712561                           0.0   
1059082 1703625                           1.0   
        1755058                           0.0   
1032311 1705264                          50.0   
1050449 1738962                           0.0   

                 working_hours_home_while_no_kids  childcare_hours_res  \
hh_id   ind_id                                                           
1049420 1687033                               0.0                  0.0   
        1662353                               0.0                  0.0   
1011033 1631191                               0.0                  4.0   
        1687630                               8.0                  0.0   
1047651 1746405                               0.0                  0.0   
        1712561                               0.0                  0.0   
1059082 1703625                              17.0                 20.0   
        1755058                               0.0                  5.0   
1032311 1705264                              50.0                 10.0   
1050449 1738962                              38.0                  0.0   

                 homeschool_hours  
hh_id   ind_id                     
1049420 1687033               0.0  
        1662353               0.0  
1011033 1631191               0.0  
        1687630               0.0  
1047651 1746405               0.0  
        1712561               0.0  
1059082 1703625              20.0  
        1755058               5.0  
1032311 1705264               0.0  
1050449 1738962               0.0

## Our first reduction operation

Calculate the mean hours spent on different activities.

In [623]:
# Define an average() function (I'm surprised there's no built-in)
def average(list):
    """Returns the average of a list.

    Args:
        list (interable): a list of numbers
    
    Returns:
        float: the average of the list of numbers
    """
    
    out = sum(list) / len(list)
    return out 

In [624]:
# Use the built-in one from statistics:
print("Mean working hours in Nov 2019 is: ", end="")
print(mean(nov_2019['working_hours']))

# And from the self-defined one:
print("and it should be the same as the results from the self-defined function. Indeed, here's the output: ")
print(average(nov_2019['working_hours']))

Mean working hours in Nov 2019 is: 17.988636363636363
and it should be the same as the results from the self-defined function. Indeed, here's the output: 
17.988636363636363


## Our second reduction operation

Calculate the mean hours spent on different activities by gender by appending the following with the appropriate method to calculate a mean.

In [625]:
nov_2019.groupby("gender")[["working_hours", "childcare_hours"]].mean()

,working_hours,childcare_hours
gender,,
Male,23.434783,7.315789
Female,12.023810,18.263158


## Append the 2 dataframes


In [639]:
# Creating new columns before appending:
apr_2020['total_working_hours'] = pd.DataFrame([apr_2020.working_hours_home_nokids, apr_2020.working_hours_home_while_no_kids, apr_2020.working_hours_home_whilekids, apr_2020.working_hours_workplace]).sum(axis=1, skipna=True)

apr_2020['total_childcare_hours'] = pd.DataFrame([apr_2020.childcare_hours_res, apr_2020.working_hours_home_whilekids]).sum(axis=1, skipna=True)

In [640]:
apr_2020.head(10)

,gender,working_hours_workplace,working_hours_home_nokids,working_hours_home_whilekids,working_hours_home_while_no_kids,childcare_hours_res,homeschool_hours,hh_id,ind_id,total_working_hours,total_childcare_hours
0,Male,0.0,NaN,0.0,0.0,0.0,0.0,1049420,1687033,NaN,NaN
1,Female,3.0,NaN,0.0,0.0,0.0,0.0,1049420,1662353,NaN,NaN
2,Male,47.0,NaN,4.0,0.0,4.0,0.0,1011033,1631191,NaN,NaN
3,Female,8.0,NaN,0.0,8.0,0.0,0.0,1011033,1687630,NaN,NaN
4,Male,36.0,NaN,0.0,0.0,0.0,0.0,1047651,1746405,NaN,NaN
5,Female,0.0,NaN,0.0,0.0,0.0,0.0,1047651,1712561,NaN,NaN
6,Female,0.0,NaN,1.0,17.0,20.0,20.0,1059082,1703625,NaN,NaN
7,Male,0.0,NaN,0.0,0.0,5.0,5.0,1059082,1755058,NaN,NaN
8,Male,40.0,NaN,50.0,50.0,10.0,0.0,1032311,1705264,NaN,NaN
9,Male,0.0,NaN,0.0,38.0,0.0,0.0,1050449,1738962,NaN,NaN


In [641]:
data_appended = apr_2020.append(nov_2019)

In [642]:
# Verify that we did not miss any row:
print(f"Total number of rows is: {apr_2020.shape[0] + nov_2019.shape[0]}")

# Print the number of rows of apr_2020:
print(f"Number of obs in Apr: {apr_2020.shape[0]}")

# Print the number of rows of nov_2019:
print(f"Number of obs in Apr: {nov_2019.shape[0]}")

Total number of rows is: 194
Number of obs in Apr: 106
Number of obs in Apr: 88


In [643]:
# Create a month column:
month = np.repeat(np.array(['Apr', 'Nov']), [106, 88])

In [644]:
# Append it to the table:
data_appended['month'] = month

In [645]:
data_appended.head()

,childcare_hours,childcare_hours_res,gender,hh_id,homeschool_hours,ind_id,total_childcare_hours,total_working_hours,working_hours,working_hours_home_nokids,working_hours_home_while_no_kids,working_hours_home_whilekids,working_hours_workplace,month
0,NaN,0.0,Male,1049420,0.0,1687033,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Apr
1,NaN,0.0,Female,1049420,0.0,1662353,NaN,NaN,NaN,NaN,0.0,0.0,3.0,Apr
2,NaN,4.0,Male,1011033,0.0,1631191,NaN,NaN,NaN,NaN,0.0,4.0,47.0,Apr
3,NaN,0.0,Female,1011033,0.0,1687630,NaN,NaN,NaN,NaN,8.0,0.0,8.0,Apr
4,NaN,0.0,Male,1047651,0.0,1746405,NaN,NaN,NaN,NaN,0.0,0.0,36.0,Apr


In [646]:
data_appended.columns.values

array(['childcare_hours', 'childcare_hours_res', 'gender', 'hh_id',
       'homeschool_hours', 'ind_id', 'total_childcare_hours',
       'total_working_hours', 'working_hours',
       'working_hours_home_nokids', 'working_hours_home_while_no_kids',
       'working_hours_home_whilekids', 'working_hours_workplace', 'month'],
      dtype=object)

In [647]:
data_appended = data_appended[['hh_id', 'ind_id', 'month', 'total_childcare_hours', 'total_working_hours', 'working_hours_workplace', 'working_hours_home_whilekids', 'working_hours_home_while_no_kids', 'working_hours_home_nokids', 'homeschool_hours', 'childcare_hours_res', 'gender']]

In [648]:
data_appended

,hh_id,ind_id,month,total_childcare_hours,total_working_hours,working_hours_workplace,working_hours_home_whilekids,working_hours_home_while_no_kids,working_hours_home_nokids,homeschool_hours,childcare_hours_res,gender
0,1049420,1687033,Apr,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,Male
1,1049420,1662353,Apr,NaN,NaN,3.0,0.0,0.0,NaN,0.0,0.0,Female
2,1011033,1631191,Apr,NaN,NaN,47.0,4.0,0.0,NaN,0.0,4.0,Male
3,1011033,1687630,Apr,NaN,NaN,8.0,0.0,8.0,NaN,0.0,0.0,Female
4,1047651,1746405,Apr,NaN,NaN,36.0,0.0,0.0,NaN,0.0,0.0,Male
5,1047651,1712561,Apr,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,Female
6,1059082,1703625,Apr,NaN,NaN,0.0,1.0,17.0,NaN,20.0,20.0,Female
7,1059082,1755058,Apr,NaN,NaN,0.0,0.0,0.0,NaN,5.0,5.0,Male
8,1032311,1705264,Apr,NaN,NaN,40.0,50.0,50.0,NaN,0.0,10.0,Male
9,1050449,1738962,Apr,NaN,NaN,0.0,0.0,38.0,NaN,0.0,0.0,Male


In [ ]:
data_appended.drop(['gender'], axis=1, inplace=True)

In [ ]:
data_apr = data_appended['month' == 'Apr'].drop_duplicates(subset=['hh_id', 'ind_id'])